In [56]:
#import numpy to use seed 
import numpy as np
import pandas as pd
from numpy.random import seed
np.random.seed(1)

In [57]:
#import other dependencies

import os
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, Dropout

Read CSV create, and clean dataframe

In [58]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("Teams.csv")

mlb_teams

,yearID,lgID,G,G.1,W,L,WSWin,R,AB,H,...,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,1997,AL,162,82,84,78,N,829,5628,1531,...,25,161,617,953,126,4.52,9,5,39,Anaheim Angels
1,1998,AL,162,81,85,77,N,787,5630,1530,...,27,147,510,1028,93,4.49,3,5,52,Anaheim Angels
2,1999,AL,162,81,70,92,N,711,5494,1404,...,22,158,511,1022,71,4.79,4,7,37,Anaheim Angels
3,2000,AL,162,81,82,80,N,864,5628,1574,...,34,236,608,1024,93,5.00,5,3,46,Anaheim Angels
4,2001,AL,162,81,75,87,N,691,5551,1447,...,26,158,494,1001,116,4.20,6,1,43,Anaheim Angels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,2017,NL,162,81,97,65,N,819,5553,1477,...,31,215,542,1327,108,3.88,3,5,46,Washington Nationals
1345,2018,NL,162,81,82,80,N,771,5517,1402,...,25,191,631,1289,119,4.04,2,7,40,Washington Nationals
1346,1969,AL,162,81,86,76,N,694,5447,1365,...,40,148,630,900,52,3.49,28,10,41,Washington Senators
1347,1970,AL,162,81,70,92,N,626,5460,1302,...,28,138,635,989,72,3.80,20,11,40,Washington Senators


In [59]:
#Get list of dataframe columns

mlb_teams.columns

Index(['yearID', 'lgID', 'G', 'G.1', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name'],
      dtype='object')

In [60]:
#Select columns for dataframe


mlb_df = mlb_teams[['yearID', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]
mlb_df.head(20)

,yearID,W,L,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,1997,84,78,N,829,5628,1531,279,25,161,617,953,126,4.52,9,5,39,Anaheim Angels
1,1998,85,77,N,787,5630,1530,314,27,147,510,1028,93,4.49,3,5,52,Anaheim Angels
2,1999,70,92,N,711,5494,1404,248,22,158,511,1022,71,4.79,4,7,37,Anaheim Angels
3,2000,82,80,N,864,5628,1574,309,34,236,608,1024,93,5.00,5,3,46,Anaheim Angels
4,2001,75,87,N,691,5551,1447,275,26,158,494,1001,116,4.20,6,1,43,Anaheim Angels
5,2002,99,63,Y,851,5678,1603,333,32,152,462,805,117,3.69,7,14,54,Anaheim Angels
6,2003,77,85,N,736,5487,1473,276,33,150,476,838,129,4.28,5,9,39,Anaheim Angels
7,2004,92,70,N,836,5675,1603,272,37,162,450,942,143,4.28,2,11,50,Anaheim Angels
8,1998,65,97,N,665,5491,1353,235,46,159,489,1239,73,4.63,7,6,37,Arizona Diamondbacks
9,1999,100,62,N,908,5658,1566,289,46,216,588,1045,137,3.77,16,9,42,Arizona Diamondbacks


In [61]:
#Use getdummies to convert strings

data_LG = mlb_df.copy()
data_bin = pd.get_dummies(data_LG, columns=["WSWin"])
data_bin = data_bin.drop(columns=['WSWin_N'])

data_bin.head()

,yearID,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name,WSWin_Y
0,1997,84,78,829,5628,1531,279,25,161,617,953,126,4.52,9,5,39,Anaheim Angels,0
1,1998,85,77,787,5630,1530,314,27,147,510,1028,93,4.49,3,5,52,Anaheim Angels,0
2,1999,70,92,711,5494,1404,248,22,158,511,1022,71,4.79,4,7,37,Anaheim Angels,0
3,2000,82,80,864,5628,1574,309,34,236,608,1024,93,5.00,5,3,46,Anaheim Angels,0
4,2001,75,87,691,5551,1447,275,26,158,494,1001,116,4.20,6,1,43,Anaheim Angels,0


In [62]:
#Set X(data) and y(team)variables

data = data_bin.drop("name", axis=1)
outcome = data_bin["name"]
print(data.shape, outcome.shape)

(1349, 17) (1349,)


In [63]:
data_bin.count()

yearID     1349
W          1349
L          1349
R          1349
AB         1349
H          1349
2B         1349
3B         1349
HR         1349
BB         1349
SO         1349
SB         1349
ERA        1349
CG         1349
SHO        1349
SV         1349
name       1349
WSWin_Y    1349
dtype: int64

Preprocess data

In [64]:
#Import SciKit libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical

In [65]:
#Split data and teams into training and test data

X_train, X_test, y_train, y_test = train_test_split(data, outcome, random_state=42)
X_train

,yearID,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,WSWin_Y
945,1976,101,61,770,5528,1505,259,45,110,542,793,127,3.08,34,9,44,0
1022,2005,67,95,680,5573,1445,292,38,139,471,1092,73,4.42,4,14,35,0
842,1971,82,80,648,5413,1377,195,43,97,581,717,75,3.43,67,15,12,0
342,2011,79,83,735,5612,1438,264,19,183,535,1250,97,4.16,4,5,39,0
354,1973,71,91,680,5592,1429,205,29,158,471,793,60,4.58,55,9,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1979,71,91,672,5395,1328,192,36,125,580,925,140,4.16,25,6,34,0
1130,2015,84,78,696,5565,1486,288,39,136,457,1159,93,3.72,7,18,41,0
1294,1980,67,95,624,5571,1398,249,53,126,448,813,67,4.19,39,9,23,0
860,1989,74,87,698,5458,1470,229,23,130,502,831,137,4.50,15,9,44,0


In [66]:
#Scale data using MinMaxScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[-1.30061815,  1.73056282, -1.61286006, ..., -0.18482387,
         0.64334638, -0.1893691 ],
       [ 0.70838156, -1.09552708,  1.26447291, ...,  1.00081658,
        -0.34916103, -0.1893691 ],
       [-1.64699741,  0.15127729, -0.00493869, ...,  1.23794467,
        -2.88556884, -0.1893691 ],
       ...,
       [-1.02351474, -1.09552708,  1.26447291, ..., -0.18482387,
        -1.67250424, -0.1893691 ],
       [-0.40003207, -0.51368504,  0.58745339, ..., -0.18482387,
         0.64334638, -0.1893691 ],
       [ 1.12403668,  0.48375845, -0.34344845, ...,  0.5265604 ,
         1.52557518, -0.1893691 ]])

In [67]:
#Label-encode team

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_test

array([ 7, 26, 13,  8, 26, 20,  8, 19, 13,  2, 10, 33, 23, 14, 29,  2,  7,
        3, 14, 26,  3,  2,  9, 26, 25, 16,  9,  8, 25, 22, 25,  9, 25,  9,
        6,  2,  7,  4, 22, 23, 26,  2,  7,  5, 23,  2, 23, 14, 23, 32,  4,
       22,  6, 24, 16,  3,  7, 19, 19, 33, 23, 11, 26,  9,  6,  5, 19, 25,
       29,  2, 27, 24, 14, 22,  2, 16,  6, 22, 26, 23, 11, 11,  7, 27, 13,
       11, 21, 26, 21, 26, 32, 13, 10, 30, 27, 19, 25, 12, 10, 11, 18, 16,
       20, 19, 26, 21, 33, 14,  5,  6, 14,  2, 28, 20,  9,  8,  6, 23, 14,
       23, 22, 26,  2, 10, 30,  6, 16,  1, 18,  8, 18, 15, 22, 19, 11, 18,
       20, 22,  8,  6, 29,  3, 27, 11,  9, 12, 32, 30, 21, 20, 13, 26, 13,
       24, 21, 32,  4,  6,  7, 23, 25, 26, 32, 23, 23,  7, 22, 28, 16, 27,
        2, 18, 32, 14, 31, 14, 27,  3,  3, 32, 24,  8, 24, 34, 16, 32, 22,
       22,  4, 13, 27, 25, 29, 28, 11, 28,  4,  9, 28, 28, 26, 33, 33, 21,
        2, 24, 18,  6, 26,  2,  9,  6,  8, 32, 23, 30,  9, 33,  2, 13, 27,
       24,  6, 26, 11, 15

In [68]:
#One hot_encode the team data

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
for x in y_test_categorical:
    print(x)

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[

Create Neural Network Model

In [79]:
# Create model and add layers
model = Sequential()
model.add(Dense(100, input_dim=17, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(45, activation='sigmoid'))
model.add(Dense(36, activation='softmax'))

In [80]:
#View model summary

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               1800      
_________________________________________________________________
dense_7 (Dense)              (None, 36)                3636      
Total params: 5,436
Trainable params: 5,436
Non-trainable params: 0
_________________________________________________________________


Train Model

In [81]:
#Compile model
from tensorflow.keras.optimizers import SGD

opt = SGD(lr=0.01)

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [82]:
print(outcome.shape)

(1349,)


In [83]:
#Fit data to train model

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
32/32 - 0s - loss: 0.1266 - accuracy: 0.0584
Epoch 2/100
32/32 - 0s - loss: 0.1210 - accuracy: 0.1009
Epoch 3/100
32/32 - 0s - loss: 0.1173 - accuracy: 0.1276
Epoch 4/100
32/32 - 0s - loss: 0.1143 - accuracy: 0.1612
Epoch 5/100
32/32 - 0s - loss: 0.1118 - accuracy: 0.1602
Epoch 6/100
32/32 - 0s - loss: 0.1096 - accuracy: 0.1780
Epoch 7/100
32/32 - 0s - loss: 0.1075 - accuracy: 0.1919
Epoch 8/100
32/32 - 0s - loss: 0.1058 - accuracy: 0.1968
Epoch 9/100
32/32 - 0s - loss: 0.1042 - accuracy: 0.2077
Epoch 10/100
32/32 - 0s - loss: 0.1027 - accuracy: 0.2146
Epoch 11/100
32/32 - 0s - loss: 0.1013 - accuracy: 0.2315
Epoch 12/100
32/32 - 0s - loss: 0.1000 - accuracy: 0.2413
Epoch 13/100
32/32 - 0s - loss: 0.0988 - accuracy: 0.2552
Epoch 14/100
32/32 - 0s - loss: 0.0978 - accuracy: 0.2522
Epoch 15/100
32/32 - 0s - loss: 0.0966 - accuracy: 0.2671
Epoch 16/100
32/32 - 0s - loss: 0.0957 - accuracy: 0.2750
Epoch 17/100
32/32 - 0s - loss: 0.0947 - accuracy: 0.2948
Epoch 18/100
32/32 - 0s

Evaluate Model

In [84]:
#Evaluate model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 0s - loss: 0.1122 - accuracy: 0.1893
Deep Neural Network - Loss: 0.11224617063999176, Accuracy: 0.18934911489486694


In [87]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("MLB 2019 season.csv")

#mlb_teams

mlb_df = mlb_teams[['yearID','W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]
#mlb_df.head(20)

data_LG = mlb_df.copy()
data_bin = pd.get_dummies(data_LG, columns=["WSWin"])
data_bin = data_bin.drop(columns=['WSWin_N'])

#data_bin.head()

feature = data_bin.drop("name", axis=1)
team = data_bin["name"]
#print(data.shape, outcome.shape)

X_feature_scaler = StandardScaler().fit(feature)
X_feature_scaled = X_feature_scaler.transform(feature)
#X_test_scaled = X_scaler.transform(X_test)
#X_train_scaled

label_team_encoder = LabelEncoder()
label_team_encoder.fit(team)
encoded_y_team = label_team_encoder.transform(team)
#encoded_y_test = label_encoder.transform(y_test)
#encoded_y_test

y_team_categorical = to_categorical(encoded_y_team)
#y_test_categorical = to_categorical(encoded_y_test)

In [91]:
encoded_predictions = model.predict_classes(X_feature_scaled[:30])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Predict World Series Champs

In [92]:
print(f"Predicted classes: {prediction_labels}")


Predicted classes: ['San Diego Padres' 'Philadelphia Phillies' 'Detroit Tigers'
 'Boston Red Sox' 'Minnesota Twins' 'Chicago Cubs' 'Chicago White Sox'
 'Baltimore Orioles' 'Minnesota Twins' 'Arizona Diamondbacks'
 'New York Yankees' 'Florida Marlins' 'Atlanta Braves' 'Oakland Athletics'
 'Philadelphia Phillies' 'Cincinnati Reds' 'Atlanta Braves'
 'Baltimore Orioles' 'Baltimore Orioles' 'New York Yankees'
 'Boston Red Sox' 'Kansas City Royals' 'San Diego Padres'
 'Oakland Athletics' 'Pittsburgh Pirates' 'Oakland Athletics'
 'San Diego Padres' 'Philadelphia Phillies' 'Milwaukee Brewers'
 'St. Louis Cardinals']


In [28]:
import joblib
filename = 'SWilliams_SVM.sav'
joblib.dump(model, filename)

TypeError: can't pickle _thread.RLock objects